In [1]:
import numpy as np
import matplotlib.pyplot as plt
from data import load_training_data, load_test_data, split_data
import networkx as nx
import time, datetime
from utils import plot_graph

from n_walk import product_graph, graph_product, graph_product_el, get_labels_nodes
import kernel_class as kc
import kernel_methods as km

from importlib import reload

In [2]:
split_list, split_idx = split_data(n_splits=3, return_indices=True)
submi_graphs = load_test_data()

sub_graphs, sub_labels = split_list[0]
sub_idx = split_idx[0]

In [3]:
rwk = kc.RandomWalkKernel(lam=0.95, save_kernel=True) #norm1=True, norm2=False, exclude_intruding_nodes=True, exclude_lonely_nodes=False, fast=True, max_iter=100

In [4]:
# print(datetime.datetime.now())
# pepin = rwk_pepin.compute_gram_matrix(sub_graphs)
# print(datetime.datetime.now())
# print(datetime.datetime.now())
# palou = rwk_palou.compute_outer_gram(lower_submi, sub_graphs, save_suf="3_subsets_1submi_lower")
# now_str = datetime.datetime.now().strftime("%m%d_%H%M%S%f")
# np.save(f"saved/RandomWalkKernel_outer_3_subsets_1submi_lower_{now_str}_idx_submi.npy", lower_submi)
# np.save(f"saved/RandomWalkKernel_outer_3_subsets_1submi_lower_{now_str}_idx_train.npy", sub_graphs)
# print(datetime.datetime.now())

In [5]:
reload(km)
svm = km.KernelSVM(lmbd=1, kernel_name="RandomWalkKernel", precomputed_kernel=True, kernel_path="saved/RandomWalkKernel_3_subset_0_psd.npy", rwlam=0.95)

In [15]:
K = np.load("saved/RandomWalkKernel_3_subset_0_psd.npy")
y = sub_labels

In [16]:
K = psdwarp(K, eps=1e-1)
K = K[:400, :400]
y = y[:400]
np.linalg.eigvalsh(K).min(), np.all(K == K.T)

(0.09999999999887028, True)

In [17]:
import cvxpy as cp
alpha = cp.Variable(K.shape[0])
obj = cp.Maximize(2 * alpha.T @ y - cp.quad_form(alpha,K))

In [18]:
obj.args

ArpackNoConvergence: ARPACK error -1: No convergence (4001 iterations, 0/1 eigenvectors converged)

In [10]:
obj.is_dcp()

In [19]:
svm.fit(np.zeros((2000)), split_list[0][1])

True
1.1657257866464734e-10
[-1 -1 -1 ... -1 -1 -1]
Fitting KernelSVM


DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP. Its following subexpressions are not:
QuadForm(var69, [[ 459.90269491  443.86211335  724.24683639 ...  137.45916317
    46.21994802  182.20709308]
 [ 443.86211335  447.02398007  716.38283135 ...  138.67923818
    42.14827568  180.95553376]
 [ 724.24683639  716.38283135 1574.30053049 ...  219.72169739
    37.02166901  241.84658172]
 ...
 [ 137.45916317  138.67923818  219.72169739 ...   55.99844277
    11.96659012   56.74848878]
 [  46.21994802   42.14827568   37.02166901 ...   11.96659012
   195.07531145   30.40526717]
 [ 182.20709308  180.95553376  241.84658172 ...   56.74848878
    30.40526717  645.83643777]])